In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.AGD(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="armijo")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.06829997897148132
epoch:  1, loss: 0.05813758820295334
epoch:  2, loss: 0.05378345027565956
epoch:  3, loss: 0.04686214402318001
epoch:  4, loss: 0.034382425248622894
epoch:  5, loss: 0.030990509316325188
epoch:  6, loss: 0.029864884912967682
epoch:  7, loss: 0.029835103079676628
epoch:  8, loss: 0.028558358550071716
epoch:  9, loss: 0.02830231562256813
epoch:  10, loss: 0.02792144939303398
epoch:  11, loss: 0.02752295322716236
epoch:  12, loss: 0.027108754962682724
epoch:  13, loss: 0.02680131420493126
epoch:  14, loss: 0.02664206176996231
epoch:  15, loss: 0.02625430002808571
epoch:  16, loss: 0.02615406922996044
epoch:  17, loss: 0.02565639652311802
epoch:  18, loss: 0.025568075478076935
epoch:  19, loss: 0.025379784405231476
epoch:  20, loss: 0.02490023337304592
epoch:  21, loss: 0.024666495621204376
epoch:  22, loss: 0.02458113431930542
epoch:  23, loss: 0.02425813116133213
epoch:  24, loss: 0.02418137900531292
epoch:  25, loss: 0.02415066957473755
epoch:  26, l

In [7]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.AGD(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.3240857422351837
epoch:  1, loss: 0.21067820489406586
epoch:  2, loss: 0.17104612290859222
epoch:  3, loss: 0.06354469060897827
epoch:  4, loss: 0.051769256591796875
epoch:  5, loss: 0.046311840415000916
epoch:  6, loss: 0.042809393256902695
epoch:  7, loss: 0.032168056815862656
epoch:  8, loss: 0.029955752193927765
epoch:  9, loss: 0.02869473770260811
epoch:  10, loss: 0.028109196573495865
epoch:  11, loss: 0.027837641537189484
epoch:  12, loss: 0.027670446783304214
epoch:  13, loss: 0.02755667455494404
epoch:  14, loss: 0.027445875108242035
epoch:  15, loss: 0.027359776198863983
epoch:  16, loss: 0.02729720063507557
epoch:  17, loss: 0.027219567447900772
epoch:  18, loss: 0.027035871520638466
epoch:  19, loss: 0.02681972272694111
epoch:  20, loss: 0.026701845228672028
epoch:  21, loss: 0.02659487910568714
epoch:  22, loss: 0.02649693191051483
epoch:  23, loss: 0.026419643312692642
epoch:  24, loss: 0.026222681626677513
epoch:  25, loss: 0.026145461946725845
epoch: 